In [ ]:
import scipy.io
import matplotlib.pyplot as plt
import os
import glob
from scipy.stats import entropy
import sys
import matplotlib.pyplot as plt
import pywt
import numpy as np
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pandas as pd
import numpy as np
from scipy.signal import welch
from scipy.interpolate import interp1d

In [ ]:
"""Mounting Google Drive"""
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pip install rarfile

In [ ]:
import rarfile
import os

# Path to EEG .rar file
rar_file_path = '/content/drive/MyDrive/EEG Data 2 - Bm Files/ASD BM - 15 files.rar'
extract_path = '/tmp/EEG/ASD/'

# Extract all files from the .rar archive
with rarfile.RarFile(rar_file_path, 'r') as rar_ref:
    rar_ref.extractall(os.path.join(extract_path))
    print(f"EEG Data Successfully extracted to {extract_path}")

# Path to EEG .rar file
rar_file_path = '/content/drive/MyDrive/EEG Data 2 - Bm Files/TD BM - 15 files.rar'
extract_path = '/tmp/EEG/TD/'

# Extract all files from the .rar archive
with rarfile.RarFile(rar_file_path, 'r') as rar_ref:
    rar_ref.extractall(os.path.join(extract_path))
    print(f"EEG Data Successfully extracted to {extract_path}")

EEG Data Successfully extracted to /tmp/EEG/ASD/
EEG Data Successfully extracted to /tmp/EEG/TD/


In [ ]:
def find_mat_files(root_dir):
    mat_files = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.mat'):
                mat_files.append(os.path.join(dirpath, filename))
    return mat_files

# file_path = r'/tmp/EEG/ASD/'
file_path = r'/tmp/EEG/TD/'

mat_files = find_mat_files(file_path)

In [ ]:
# Function to perform window slicing on the EEG signal
def window_slice(signal, window_size, overlap):
    """
    Perform window slicing on the signal.

    Parameters:
    - signal: The input signal (1D array).
    - window_size: The size of each window.
    - overlap: The overlap between consecutive windows.

    Returns:
    - windows: 2D array containing sliced windows.
    """
    windows = []
    for i in range(0, len(signal) - window_size + 1, window_size - overlap):
        window = signal[i:i + window_size]
        windows.append(window)
    return np.array(windows)

In [ ]:
# Function to extract time-domain features from each window
def extract_time_domain_features(window):
    features = [
        np.mean(window),
        np.std(window),
        np.median(window),
        np.max(window) - np.min(window),
        np.mean((window - np.mean(window)) ** 3) / np.std(window) ** 3,  # Skewness
        np.mean((window - np.mean(window)) ** 4) / np.var(window) ** 2 - 3,  # Kurtosis
    ]
    return features

In [ ]:
# Function to extract frequency band power from each window
def extract_frequency_bands(window, sampling_rate):
    f, psd = welch(window, fs=sampling_rate, nperseg=len(window))

    # Define frequency bands (adjust as needed)
    delta_band = (0.5, 4)
    theta_band = (4, 8)
    alpha_band = (8, 13)
    beta_band = (13, 30)
    gamma_band = (30, 40)

    # Interpolate PSD to ensure even frequency spacing
    f_interp = np.linspace(f.min(), f.max(), len(f))
    psd_interp = interp1d(f, psd, kind='linear', fill_value='extrapolate')(f_interp)

    # Calculate index range for each frequency band
    delta_indices = np.where((f_interp >= delta_band[0]) & (f_interp <= delta_band[1]))[0]
    theta_indices = np.where((f_interp >= theta_band[0]) & (f_interp <= theta_band[1]))[0]
    alpha_indices = np.where((f_interp >= alpha_band[0]) & (f_interp <= alpha_band[1]))[0]
    beta_indices = np.where((f_interp >= beta_band[0]) & (f_interp <= beta_band[1]))[0]
    gamma_indices = np.where((f_interp >= gamma_band[0]) & (f_interp <= gamma_band[1]))[0]

    # Extract power in each frequency band
    delta_power = np.trapz(psd_interp[delta_indices], dx=f_interp[1] - f_interp[0])
    theta_power = np.trapz(psd_interp[theta_indices], dx=f_interp[1] - f_interp[0])
    alpha_power = np.trapz(psd_interp[alpha_indices], dx=f_interp[1] - f_interp[0])
    beta_power = np.trapz(psd_interp[beta_indices], dx=f_interp[1] - f_interp[0])
    gamma_power = np.trapz(psd_interp[gamma_indices], dx=f_interp[1] - f_interp[0])

    return {
        'Delta': delta_power,
        'Theta': theta_power,
        'Alpha': alpha_power,
        'Beta': beta_power,
        'Gamma': gamma_power
    }

In [ ]:
index = 0
for mat_file_path in mat_files:
  try:
    mat_data = scipy.io.loadmat(mat_file_path)
    try:
      eeg_data = mat_data['Bio_Segment4']
    except:
      try:
        eeg_data = mat_data['Known_Segment4']
      except:
        eeg_data = mat_data['EO_Segment4']

    print("shape of bio segment 1",eeg_data.shape)

    if len(mat_files) == 1:

      # Plot EEG channels
      num_channels, num_time_points = eeg_data.shape

      for channel in range(num_channels):
          plt.plot(eeg_data[channel, :], label=f'Channel {channel + 1}')

      plt.title('EEG Data Plot')
      plt.xlabel('Time Points')
      plt.ylabel('Amplitude')
      plt.legend()
      plt.show()

      # Plot the top 5 EEG channels
      num_channels, num_time_points = eeg_data.shape
      num_channels_to_plot = min(5, num_channels)  # Choose the top 5 channels or all channels if less than 5

      for channel in range(num_channels_to_plot):
          plt.plot(eeg_data[channel, :], label=f'Channel {channel + 1}')

      plt.title('Top 5 EEG Channels')
      plt.xlabel('Time Points')
      plt.ylabel('Amplitude')
      plt.legend()
      plt.show()


    # Parameters for window slicing
    window_size = 100  # Size of each window
    overlap = 50  # Overlap between consecutive windows


    # Apply window slicing to the EEG signal
    eeg_windows = window_slice(eeg_data, window_size, overlap)
    eeg_windows = np.reshape(eeg_windows, (100, 600))

    if len(mat_file_path) == 1:

      # Plot the original EEG signal and the sliced windows
      plt.figure(figsize=(10, 6))

      plt.subplot(2, 1, 1)
      plt.plot(eeg_data)
      plt.title('Reconstructed EEG Signal')
      plt.xlabel('Sample')
      plt.ylabel('Amplitude')

      plt.subplot(2, 1, 2)

      for i, window in enumerate(eeg_windows):
          plt.subplot(len(eeg_windows), 1, i + 1)
          plt.plot(window, color='b')
          plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
          plt.title(f'Window {i + 1}')
          plt.xlabel('Sample')
          plt.ylabel('Amplitude')

      # Adjust layout and save to an Excel file
      plt.tight_layout()
      plt.savefig('eeg_windows_16.png')
      plt.close()


    # Extract time-domain features for each window
    time_domain_features = [extract_time_domain_features(window) for window in eeg_windows]

    # Convert the list of features to a NumPy array
    time_domain_features_array = np.array(time_domain_features)

    # Print or use the features as needed
    print("Time-domain features shape:", time_domain_features_array.shape)
    print("Example features for the first window:", time_domain_features_array[0])


    # Save features to Excel file
    feature_names = ['Mean', 'Std', 'Median', 'Range', 'Skewness', 'Kurtosis']
    df = pd.DataFrame(time_domain_features_array, columns=feature_names)

    # Save the DataFrame to an Excel file
    folder_number = str(index)
    index += 1
    # Create Excel file name
    # excel_file_path = '/content/drive/MyDrive/Features Dataset2_ASD_TD/ASD/' + folder_number + '.xlsx'
    excel_file_path = '/content/drive/MyDrive/Features Dataset2_ASD_TD/TD/' + folder_number + '.xlsx'

    df.to_excel(excel_file_path, index=False)

    print(f'Time-domain features saved to {excel_file_path}')


    # Sample rate of the EEG signal (adjust as needed)
    sampling_rate = 1000  # Hz

    # Extract frequency band power for each window
    frequency_band_powers = [extract_frequency_bands(window, sampling_rate) for window in eeg_windows]

    # Convert the list of features to a pandas DataFrame
    frequency_band_df = pd.DataFrame(frequency_band_powers)
    # Save the DataFrame to an Excel file
    # excel_file_path = '/content/drive/MyDrive/Features Dataset2_ASD_TD/ASD/' + folder_number + '_bands.xlsx'
    excel_file_path = '/content/drive/MyDrive/Features Dataset2_ASD_TD/TD/' + folder_number + '_bands.xlsx'

    frequency_band_df.to_excel(excel_file_path, index=False)
    print(f'Frequency band powers saved to {excel_file_path}')
  except:
    print('File Failed to be Processed')
    index += 1

shape of bio segment 1 (125, 600)
Time-domain features shape: (100, 6)
Example features for the first window: [-4.28430938  9.39187038 -4.14708376 45.82211876 -0.3534539  -0.33584369]
Time-domain features saved to /content/drive/MyDrive/Features Dataset2_ASD_TD/TD/0.xlsx
Frequency band powers saved to /content/drive/MyDrive/Features Dataset2_ASD_TD/TD/0_bands.xlsx
shape of bio segment 1 (125, 600)
Time-domain features shape: (100, 6)
Example features for the first window: [-35.41410763  13.48713692 -36.33358765  59.6532402    0.29086696
  -0.50141414]
Time-domain features saved to /content/drive/MyDrive/Features Dataset2_ASD_TD/TD/1.xlsx
Frequency band powers saved to /content/drive/MyDrive/Features Dataset2_ASD_TD/TD/1_bands.xlsx
shape of bio segment 1 (125, 600)
Time-domain features shape: (100, 6)
Example features for the first window: [-13.10446718  13.04084074 -17.09512615  56.57668877   0.42798355
  -0.88881815]
Time-domain features saved to /content/drive/MyDrive/Features Datase